In [ ]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import math
#import scienceplots
#plt.style.use(['science','nature','no-latex','vibrant'])
from scipy import constants

#Import personal functions
import sys
#sys.path.append("C:/Users/Olivi/Documents/Git/SoRoMa/Thermal_Soft_Robotics/Functions")
#sys.path.append("C:/Users/Olivi/OneDrive - TU Eindhoven/Soft robotic matter/GitHub/Thermal_Soft_Robotics/Functions")
sys.path.append("//sun/overvelde/group-folder/PROJECTS/Thermal_soft_robotics_Olivier/Git/Thermal_Soft_Robotics/Functions")
import FluidicSetupFunctions
from save_plot_func import save_plot
#import main_extended_functions 
from scipy.integrate import solve_ivp

from main_extended_functions import *

In [ ]:
# Novec properties
c_n = 1300 # Specific heat capacity novec [J/kg/K] source: datasheet
M_n = 200e-3 # Molar mass novec [kg/mol] source: datasheet
m_n = 2e-3 # Mass novec [kg] source: datasheet
n_n = m_n/M_n # Amount of novec in pouches [mol]
boiling_point = 34+273.15 # Boiling point novec [K] source: datasheet
latent_heat = 142e3 # Latent heat of vaporization novec [J/kg] source: datasheet
K_n = 0.075 # Thermal conductivity novec [W/m/K] source: datasheet
nu_n_g = 0.32e-3 # Kinematic viscosity novec gas [m^2/s] source: datasheet 


In [ ]:
# Make dictionary of all constants 
# System parameters in a dictionary
params = {
    # novec properties
    "c_n": c_n,
    "M_n": M_n,
    "m_n": m_n, 
    "latent_heat": latent_heat,
    "boiling_point": boiling_point,
    "K_n": K_n,
    "nu_n_g": nu_n_g,
    "n_n": n_n,
    
}

In [ ]:
def system(t, y, **params):
    T_n, n_g, n_l = y
  
    # Unpack parameters for use in the function
    

    # Novec properties
    c_n = params['c_n']
    M_n = params['M_n']
    m_n = params['m_n']
    latent_heat = params['latent_heat']
    boiling_point = params['boiling_point']
    nu_n_g = params['nu_n_g']
    n_n = params['n_n']

    
    # Model T_n and boiling + evaporation
    dT_n_dt = 100/(m_n*c_n)
    dn_g_dt = 0
    dn_l_dt = 0
    
   
    '''
    if T_n >= boiling_point and n_l >1e-10:
        # Convert evaporated mass to moles and add to the gas in the vessel
        Q_available = m_n*c_n*(T_n - boiling_point)
        dT_n_dt=- Q_available/(n_l*c_n*M_n)

        evaporation_rate = min(Q_available / latent_heat,(n_l*M_n)/ )  * M_n
        
        #print(evaporated_moles,n_l)
       
        dn_l_dt =- evaporation_rate        
        dn_g_dt =+ evaporation_rate
    '''    
    


    print('t = ', t)
    
    return [ dT_n_dt, dn_g_dt, dn_l_dt]
      

In [ ]:
# Define initial conditions
gamma0 = 0.5
y0 = [ 273+20, gamma0*n_n,(1-gamma0)* n_n]
print(y0)


t = np.linspace(0, 24*3600, 24*3600)
sol = solve_ivp(
    lambda t, y: system(t, y, **params),  # Unpack params here
    [t[0], t[-1]],
    y0,
    dense_output=True,
    method='LSODA',
    t_eval=np.linspace(t[0], t[-1], 24*3600),
    rtol=1e-6,
    atol=1e-9,
    max_step=1
)
print(sol.message)
t= sol.t
y= sol.y   